<a href="https://colab.research.google.com/github/MarcLefevreFr/OpenClassrooms/blob/master/P6/P6_TL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfert Learning_01**

### **Préliminaires**<br>**Import :**<br>- google drive<br>- données<br>- bibliothèques python

In [1]:
# instal google drive
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# récupe des zip d'images et extractions
import shutil
import os
import zipfile
from time import time

start = time()
cwd = os.getcwd()

shutil.copy(os.path.join("drive/MyDrive/P6/Data/", "test.zip"), cwd)
shutil.copy(os.path.join("drive/MyDrive/P6/Data/", "train.zip"), cwd)

# données "low" = image originales mais juste les 30 première classes
shutil.copy(os.path.join("drive/MyDrive/P6/Data/", "test_light.zip"), cwd)
shutil.copy(os.path.join("drive/MyDrive/P6/Data/", "train_light.zip"), cwd)



with zipfile.ZipFile(cwd + "/test.zip", 'r') as zip_ref :
    zip_ref.extractall(cwd)

with zipfile.ZipFile(cwd + "/train.zip", 'r') as zip_ref :
    zip_ref.extractall(cwd)

with zipfile.ZipFile(cwd + "/test_light.zip", 'r') as zip_ref :
    zip_ref.extractall(cwd)

with zipfile.ZipFile(cwd + "/train_light.zip", 'r') as zip_ref :
    zip_ref.extractall(cwd)

print(f"Import & extraction des data : {time()-start:.0f} secondes.")

Import & extraction des data : 84 secondes.


In [3]:
# vérif
import os
# chemin des répertoires contenant les photos
PATH_TRAIN = "train"
PATH_TRAIN_l = "train_light"
PATH_TEST = "test"
PATH_TEST_l = "test_light"

len(os.listdir(PATH_TRAIN)), len(os.listdir(PATH_TEST)), len(os.listdir(PATH_TRAIN_l)), len(os.listdir(PATH_TEST_l))

(120, 120, 30, 30)

In [4]:
# Import python
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import PIL.ImageOps as ops
import os
import pickle
import random as rd
from time import time

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [5]:
def modelize(model, nb_epochs, train_gen, val_gen, train_steps, val_steps, mod_name, verbose = 0):

    model.compile(loss='categorical_crossentropy',
                  optimizer='RMSprop',
                  metrics=['accuracy'])

    start = time()

    history = model.fit(train_gen,
                        steps_per_epoch = train_steps,
                        epochs = nb_epochs,
                        validation_data = val_gen,
                        validation_steps = val_steps,
                        verbose = verbose)
    
    model.save_weights("drive/MyDrive/P6/Data/models/" + mod_name + ".h5")
    pickle.dump(history.history, open("drive/MyDrive/P6/Data/models/" + mod_name + "_hist.pickle", "wb" ))

    fig = plt.figure(figsize = (10,4))

    train_acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    
    plt.plot(range(1, nb_epochs+1), train_acc, 'b', label='Train acc', color = "red")
    plt.plot(range(1, nb_epochs+1), val_acc, 'b', label='Validation acc', color = "blue")
    plt.title('Train & validation accuracies - ' + mod_name)
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    print(f"Temps d'entrainements pour {nb_epochs} epochs : {time()-start:.0f} secondes.")
    print(f"Meilleure train_acc = {np.max(train_acc):.2f}, meilleure val_acc = {np.max(val_acc):.2f}")
    print()

    plt.show()

def create_generator(batch, train_folder, size = (100, 100)):

    generators = []

    train_datagen = ImageDataGenerator(rescale = 1./255#, # normalisation
                                    #rotation_range = 15,
                                    #horizontal_flip=True,
                                    #validation_split = 0.2
                                    )

    val_datagen = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 10,
                                    zoom_range = 0.1,
                                    fill_mode = "reflect"
                                    )

    train_gen = train_datagen.flow_from_directory(train_folder,
                                                target_size = size,
                                                batch_size = batch,
                                                class_mode = "categorical",
                                                seed = SEED,
                                                interpolation = "nearest"
                                                )

    val_gen = val_datagen.flow_from_directory(train_folder,
                                            target_size = size,
                                            batch_size = batch,
                                            class_mode = "categorical",
                                            seed = SEED,
                                            interpolation = "nearest"
                                            )
    
    return train_gen, val_gen


def create_test_generators(batch, test_folder, size = (100, 100)):

    # générateur d'images test, juste la normalisation
    test_datagen = ImageDataGenerator(rescale = 1./255)

    test_gen = train_datagen.flow_from_directory(test_folder,
                                                target_size = size,
                                                batch_size = batch,
                                                class_mode = "categorical",
                                                seed = SEED,
                                                interpolation = "nearest"
                                                )
    
    return test_gen

    # RECUP DU DICO DES CLASSES :
    # ---> mon_generator.class_indices

## **Transfert**

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [9]:
# import vgg16 sans les couches denses
model = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 1s 0us/step


In [10]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     